In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Importing Required Libraries

In [ ]:
from skimage.io import imread, imshow
from skimage import transform
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import pandas as pd
import os
import random as rnd
# importing matplotib to plot images and graphs
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import torch.nn.functional as F
from skimage.color import rgb2gray
from sklearn.decomposition import PCA
import random
import numpy as np

Plotting images of each class

In [ ]:
%matplotlib inline
dataset_dir = '../input/plant-seedlings-classification/train'
subplot_size = len([dir for dir in os.listdir(dataset_dir)])
fig, axs = plt.subplots(subplot_size, sharey = True, figsize=(50, 50))

for ax, dir in zip(axs ,os.listdir(dataset_dir)):
    
    file_name = rnd.choice([file for file in os.listdir(os.path.join(dataset_dir, dir))])
#     print(file_name)
    img = mpimg.imread(os.path.join(dataset_dir, dir, file_name))
    img = transform.resize(img, (100, 100))
    ax.imshow(img)
    # Add title and axis names
    ax.set_title(dir)
    

In [ ]:
class_map = {}
id = 0
for dir_name in os.listdir('../input/plant-seedlings-classification/train'):
    class_map[dir_name] = id
    id += 1
class_reverse_map ={}
id = 0
for dir_name in os.listdir('../input/plant-seedlings-classification/train'):
    class_reverse_map[id] = dir_name
    id += 1


Applying PCA to a subset of images in training directory

Counting num. of image files in each directory

In [ ]:
train_dir = '../input/plant-seedlings-classification/train'
train_size = 0.85
sub_dir = os.listdir(train_dir)
# print(len(sub_dir))
sub_dir_lengths ={}
train_set_files = []
test_set_files = []
for sub in sub_dir:
    full_path = os.path.join(train_dir, sub)
    files = [os.path.join(train_dir, sub,name) for name in os.listdir(full_path)]
    sub_dir_lengths[sub]=len(files)
    random.shuffle(files)
    train_set_files.extend(files[:int(train_size*sub_dir_lengths[sub])])
    test_set_files.extend(files[int(train_size*sub_dir_lengths[sub]):])

In [ ]:
len(test_set_files)

In [ ]:
# for sub in os.listdir(train_dir):
#     full_path = os.path.join(train_dir, sub)
#     print(f'{sub} - ({len(train_set_files[sub])}, {len(test_set_files[sub])})')

In [ ]:
pca = PCA(n_components = 100)
# train_df = pd.DataFrame(['file_path','image'])
train_images = []
# for sub_name in train_set_files.keys():
for file_name in train_set_files:
    img = imread(file_name)
    img = transform.resize(img, (100, 100, 3))
    img = img.reshape(-1)
    train_images.append(img)
# train_df=pd.DataFrame(train_dict, columns = ['file_path','image'])
# test_dict = []
# for sub_name in test_set_files.keys():
#     for file_name in test_set_files[sub_name]:
#         img = imread(file_name)
#         img = transform.resize(img, (100, 100, 3))
#         img = img.reshape(-1)
#         test_dict.append((file_name, img))
# test_df = pd.DataFrame(test_dict, columns = ['file_path','image'])

In [ ]:
len(train_images)

fit PCA instance on training dataset

In [ ]:
images_fit = pd.DataFrame(train_images)
images_fit = images_fit.values.tolist()
pca.fit(images_fit)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
class SeedDataSet(Dataset):
    def __init__(self, dir, transform = None):
        """
        dir has all the file names from training set which will be used for training (as opposed to few for validation)
        """
        self.df = pd.DataFrame(dir, columns = ['Name'])
       #         self.df = pd.DataFrame(columns = ['Name','Labels'])
#         for i, dir_name in enumerate(os.listdir(dir)):
#             for name in os.listdir(os.path.join(dir, dir_name)):
#                 labels.append(( name, i ))
#         self.df = pd.DataFrame(labels, columns = ['Name','Labels'])
#         self.transforms = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        image_path_name = self.df.iloc[idx, 0]
        
        head_tail = os.path.split(image_path_name)
        sub_dir_name = os.path.split(head_tail[0])[1]
        
#         label = self.df.iloc[idx, 1]
#         path = os.path.join(self.dir, class_reverse_map[label] ,image_name)
        img = imread(image_path_name)
        
        img = transform.resize(img, (100, 100, 3))
        img = img.reshape(1, -1)
        img = pca.transform(img)
#         img = torch.from_numpy(img).unsqueeze(0)
        return img, class_map[sub_dir_name]

In [ ]:
train_dataset = SeedDataSet(train_set_files,transform=transforms.Resize(100))
validation_dataset = SeedDataSet(test_set_files,transform=transforms.Resize(100))

In [ ]:
len(validation_dataset)

In [ ]:
image, label = train_dataset[0]

In [ ]:
# dir = '../input/plant-seedlings-classification/train'
# shapes = []
# df = pd.DataFrame(columns = ['Name','Labels'])
# for i, dir_name in enumerate(os.listdir(dir)):
#     for name in os.listdir(os.path.join(dir, dir_name)):
#         img = imread(os.path.join(dir, dir_name,name))
#         if img.ndim > 3:
#             print(name)
# df = pd.DataFrame(labels, columns = ['Name','Labels'])

In [ ]:
# train_size = int(0.8 * len(dataset))
# test_size = len(dataset) - train_size
# train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

Image dimensions

In [ ]:
image.shape

In [ ]:
train_loader = DataLoader(train_dataset, batch_size = 100, shuffle = True)
val_loader = DataLoader(validation_dataset, batch_size = 100)

Defining a NN Model

In [ ]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
#         self.conv_layer1 = torch.nn.Sequential(
#             torch.nn.Conv2d(in_channels = 1, out_channels= 10, kernel_size = 11), 
#             torch.nn.ReLU(),
#             torch.nn.MaxPool2d(kernel_size =2, stride = 2)
#         )
#         torch.nn.init.xavier_uniform_(self.conv_layer1.weight)
#         torch.nn.init.xavier_uniform_(self.conv_layer1.bias)
        
#         self.conv_layer2 = torch.nn.Sequential(
#         torch.nn.Conv2d(in_channels = 10, out_channels = 20, kernel_size = 10), 
#         torch.nn.ReLU(),
#         torch.nn.MaxPool2d(kernel_size = 2, stride = 2)
#         )
#         torch.nn.init.xavier_uniform_(self.conv_layer2.weight)
#         torch.nn.init.xavier_uniform_(self.conv_layer2.bias)
        
        self.fc1 = torch.nn.Linear(100, 25)
        self.fc2 = torch.nn.Linear(25, 12)
        
    def forward(self, x):
#         out = self.conv_layer1(x)
#         out = self.conv_layer2(out)
#         out = out.contiguous().view(out.size(0), -1)
        out = F.relu(self.fc1(x))
        out = self.fc2(out)
        return out

In [ ]:
# defining the model
model = CNN()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

In [ ]:
learning_rate = 0.01
loss_fn = torch.nn.CrossEntropyLoss()    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, epoch):
    size = len(dataloader.dataset)
    epoch_loss = 0
    correct = 0
    num_batches = len(dataloader)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
#         X = X.permute(0,1,2)
        X = X.float()
        if torch.cuda.is_available():
            X = X.cuda()
            y = y.cuda()
        pred = model(X)
        pred = pred.squeeze()
        y = y.long()
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        
        
        
        loss, current = loss.item(), batch * len(X)
        
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        epoch_loss += loss/num_batches
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    train_loss[epoch] = epoch_loss
    train_acc[epoch] = correct
    

def test_loop(dataloader, model, loss_fn, epoch):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
#             X = X.permute(0,1,2)
            X = X.float()
            if torch.cuda.is_available():
                X = X.cuda()
                y = y.cuda()
            pred = model(X)
            pred = pred.squeeze()
            y = y.long()
#             pred = F.one_hot(pred)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    val_loss[epoch] = test_loss
    correct /= size
    val_acc[epoch] = correct
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
if torch.cuda.is_available():
    loss_fn = loss_fn.cuda()

epochs = 10
train_loss, val_loss, train_acc, val_acc = [0]*epochs, [0]*epochs, [0]*epochs, [0]*epochs
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer,  t)
    test_loop(val_loader, model, loss_fn,t)
print("Done!")

Plotting loss graphs

In [ ]:
x1 = list(range(epochs))
y1 = train_loss
y2 = val_loss
fig, (ax0, ax1) = plt.subplots(2, sharex= True)
ax0.plot(x1, y1)
ax0.plot(x1, y2)
ax0.legend(['training loss','validation loss'])
y1 = train_acc
y2 = val_acc
ax1.plot(x1, y1)
ax1.plot(x1, y2)
ax1.legend(['training accuracy','validation accuracy'])



In [ ]:
class TestDataset(Dataset):
    def __init__(self, dir):
        self.dir = dir
        
        names = [name for name in os.listdir(dir)]
        self.df = pd.DataFrame(names, columns = ['Name'])
    def __len__(self):
        return len([name for name in os.listdir(self.dir)])
    def __getitem__(self,idx):
        
        img_name = self.df.iloc[idx,0]
        image = imread(os.path.join(self.dir, img_name))
        
#         img = imread(image_path_name)
        
        img = transform.resize(image, (100, 100, 3))
        img = img.reshape(1, -1)
        img = pca.transform(img)
        
        
        return img, img_name

In [ ]:
test_Dataset=  TestDataset('../input/plant-seedlings-classification/test')

In [ ]:
test_loader = DataLoader(test_Dataset, batch_size = 1)

In [ ]:
pred_dict = []
for i, (img, img_name) in enumerate(test_loader):
    img_name = img_name[0]
    
    with torch.no_grad():
        img = img.float()
#         img = img.cuda()
        output = model(img)

        output = torch.argmax(output).cpu().numpy().item()
        pred_dict.append((img_name, class_reverse_map[output]))
        
df = pd.DataFrame(pred_dict, columns = ['file','species'])

In [ ]:
df.to_csv('sample_submission.csv', index = False)